In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import re
import openpyxl
import sklearn
from keras.utils import to_categorical

2023-04-24 15:48:43.234820: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Read In Data

In [2]:
data = pd.read_excel('apple_full_data_clean.xlsx')

In [3]:
pd.read_excel('apple_full_data_clean.xlsx')['phone model'].unique()

array(['apple iphone 11', 'apple iphone 12', 'apple iphone 13',
       'apple iphone 14', 'apple iphone 3gs', 'apple iphone 4',
       'apple iphone 4s', 'apple iphone 5', 'apple iphone 5s',
       'apple iphone 6', 'apple iphone 6s', 'apple iphone 7',
       'apple iphone 8', 'apple iphone se', 'apple iphone x',
       'apple iphone xr', 'apple iphone xs'], dtype=object)

In [4]:
data.drop(['phone color', 
    'week_of_month',
    'month', 'phone size', 'phone model', 'year', 'day'], axis = 1, inplace = True)

# One hot encode generation due to iPhones such as XS and XR
one_hot = pd.get_dummies(data['generation'])
data = data.drop('generation', axis = 1)
data = data.join(one_hot)

data[[
    'claim',
    'weeks_since_release',
    'is_holiday'
]] = data[['claim','weeks_since_release', 'is_holiday']].apply(np.float32)

test_time = [
    '2023-02-27',
    '2023-02-20',
    '2023-02-13',
    '2023-02-06',
    '2023-01-30',
    '2023-01-23',
    '2023-01-16',
    '2023-01-09',
    '2023-01-02',
    '2022-12-26',
    '2022-12-19'
]

testing_data = data.loc[data['weeks_monday'].isin(test_time)]
training_data = data.loc[~data['weeks_monday'].isin(test_time)]

### Update training data format

In [5]:
training_time = pd.to_datetime(training_data['weeks_monday'], format='%Y-%m-%d')
training_time = np.array(training_time)

training_data.drop('weeks_monday', axis = 1, inplace = True)

print(training_data.dtypes)

claim                  float32
weeks_since_release    float32
is_holiday             float32
11                       uint8
12                       uint8
13                       uint8
14                       uint8
3gs                      uint8
4                        uint8
4s                       uint8
5                        uint8
5s                       uint8
6                        uint8
6s                       uint8
7                        uint8
8                        uint8
se                       uint8
x                        uint8
xr                       uint8
xs                       uint8
dtype: object


/var/folders/qc/xhd08p257qs0hgklv6c67x640000gn/T/ipykernel_82840/2658106679.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.drop('weeks_monday', axis = 1, inplace = True)


### Update testing data format

In [6]:
testing_time = pd.to_datetime(testing_data['weeks_monday'], format='%Y-%m-%d')
testing_time = np.array(testing_time)

testing_data.drop('weeks_monday', axis = 1, inplace = True)

print(testing_data.dtypes)

claim                  float32
weeks_since_release    float32
is_holiday             float32
11                       uint8
12                       uint8
13                       uint8
14                       uint8
3gs                      uint8
4                        uint8
4s                       uint8
5                        uint8
5s                       uint8
6                        uint8
6s                       uint8
7                        uint8
8                        uint8
se                       uint8
x                        uint8
xr                       uint8
xs                       uint8
dtype: object


/var/folders/qc/xhd08p257qs0hgklv6c67x640000gn/T/ipykernel_82840/197915255.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_data.drop('weeks_monday', axis = 1, inplace = True)


In [7]:
y_train = training_data['claim']
X_train = training_data.drop(['claim'], axis=1)

y_test = testing_data['claim']
X_test = testing_data.drop(['claim'], axis=1)

In [8]:
y_train

0        111.0
1        319.0
2        320.0
3        305.0
4        269.0
         ...  
17446     26.0
17447     27.0
17448     17.0
17449     21.0
17450     18.0
Name: claim, Length: 15873, dtype: float32

In [9]:
X_train.head(), y_train.head()

(   weeks_since_release  is_holiday  11  12  13  14  3gs  4  4s  5  5s  6  6s  \
 0            92.428574         1.0   1   0   0   0    0  0   0  0   0  0   0   
 1            93.428574         1.0   1   0   0   0    0  0   0  0   0  0   0   
 2            94.428574         1.0   1   0   0   0    0  0   0  0   0  0   0   
 3            95.428574         1.0   1   0   0   0    0  0   0  0   0  0   0   
 4            96.428574         1.0   1   0   0   0    0  0   0  0   0  0   0   
 
    7  8  se  x  xr  xs  
 0  0  0   0  0   0   0  
 1  0  0   0  0   0   0  
 2  0  0   0  0   0   0  
 3  0  0   0  0   0   0  
 4  0  0   0  0   0   0  ,
 0    111.0
 1    319.0
 2    320.0
 3    305.0
 4    269.0
 Name: claim, dtype: float32)

## Build the Model

You've seen these layers before and here is how it's looks like when combined.

In [10]:
from sklearn.preprocessing import StandardScaler

# Create a scaler object
scaler = StandardScaler()

# Scale the input data and target variable
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler.fit_transform(y_test.values.reshape(-1, 1))

In [11]:
X_train_scaled

array([[-0.9251061 ,  2.1960914 ,  2.1985261 , ..., -0.13277976,
        -0.27353963, -0.26912367],
       [-0.9163754 ,  2.1960914 ,  2.1985261 , ..., -0.13277976,
        -0.27353963, -0.26912367],
       [-0.90764475,  2.1960914 ,  2.1985261 , ..., -0.13277976,
        -0.27353963, -0.26912367],
       ...,
       [ 0.17496155, -0.45535448, -0.45485017, ..., -0.13277976,
        -0.27353963,  3.715764  ],
       [ 0.18369225, -0.45535448, -0.45485017, ..., -0.13277976,
        -0.27353963,  3.715764  ],
       [ 0.19242294, -0.45535448, -0.45485017, ..., -0.13277976,
        -0.27353963,  3.715764  ]], dtype=float32)

In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(
    filters=64,
    kernel_size=3,
    activation="relu",
    input_shape=[19, 1]
  ),
  tf.keras.layers.LSTM(128, return_sequences=True),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.LSTM(128),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

 # Print the model summary 
model.summary()

2023-04-24 15:49:18.234148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 17, 64)            256       
                                                                 
 lstm (LSTM)                 (None, 17, 128)           98816     
                                                                 
 dropout (Dropout)           (None, 17, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

In [146]:

X_train_scaled

array([[-0.9251061 ,  2.1960914 ,  2.1985261 , ..., -0.13277976,
        -0.27353963, -0.26912367],
       [-0.9163754 ,  2.1960914 ,  2.1985261 , ..., -0.13277976,
        -0.27353963, -0.26912367],
       [-0.90764475,  2.1960914 ,  2.1985261 , ..., -0.13277976,
        -0.27353963, -0.26912367],
       ...,
       [ 0.17496155, -0.45535448, -0.45485017, ..., -0.13277976,
        -0.27353963,  3.715764  ],
       [ 0.18369225, -0.45535448, -0.45485017, ..., -0.13277976,
        -0.27353963,  3.715764  ],
       [ 0.19242294, -0.45535448, -0.45485017, ..., -0.13277976,
        -0.27353963,  3.715764  ]], dtype=float32)

In [13]:
# Set the training parameters
model.compile(
    loss='mse',
    optimizer='adam'
)


In [26]:
# Train the model
history = model.fit(X_train, y_train, epochs = 150, batch_size = 20)

Epoch 1/150
794/794 [==============================] - 15s 18ms/step - loss: 3446.9607
Epoch 2/150
794/794 [==============================] - 14s 17ms/step - loss: 3452.8486
Epoch 3/150
794/794 [==============================] - 14s 18ms/step - loss: 3453.6033
Epoch 4/150
794/794 [==============================] - 14s 17ms/step - loss: 3454.0679
Epoch 5/150
794/794 [==============================] - 14s 17ms/step - loss: 3453.3337
Epoch 6/150
794/794 [==============================] - 14s 17ms/step - loss: 3456.8262
Epoch 7/150
794/794 [==============================] - 14s 17ms/step - loss: 3446.0068
Epoch 8/150
794/794 [==============================] - 14s 17ms/step - loss: 3457.5269
Epoch 9/150
794/794 [==============================] - 14s 17ms/step - loss: 3452.7366
Epoch 10/150
794/794 [==============================] - 14s 17ms/step - loss: 3452.7263
Epoch 11/150
794/794 [==============================] - 15s 19ms/step - loss: 3442.8464
Epoch 12/150
794/794 [===================

In [27]:
y_pred = model.predict(X_test)
y_pred

82/82 [==============================] - 1s 8ms/step


array([[76.72621],
       [76.72621],
       [76.72621],
       ...,
       [45.61362],
       [45.61362],
       [45.61362]], dtype=float32)

In [28]:
def wmape(actual, forecast):
    """Calculate Weighted Mean Absolute Percentage Error (WMAPE) with equal weights"""
 
    numerator = np.sum(np.abs(actual - forecast)) * 100
    denominator = np.sum(np.abs(actual))

    return numerator/denominator

In [29]:
np.array(y_test), y_pred.flatten()

(array([218., 252., 233., ...,  11.,  17.,  12.], dtype=float32),
 array([76.72621, 76.72621, 76.72621, ..., 45.61362, 45.61362, 45.61362],
       dtype=float32))

In [30]:
wmape(np.array(y_test), y_pred.flatten())

83.93512287684086